In [1]:
import os
import urllib.request
import firebase_admin
from selenium import webdriver
from firebase_admin import credentials
from firebase_admin import firestore_async
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred)
db = firestore_async.client()

In [6]:
documts = db.collection('character_threads')
doc_ref = documts.stream()

opt = webdriver.ChromeOptions()
opt.add_argument("--incognito")
opt.add_argument("disable-infobars")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-gpu")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=opt)

async for doc in doc_ref:
    d = doc.to_dict()
    
    driver.get(d.get('link'))
    wait = WebDriverWait(driver, 20)
    
    wait.until(EC.visibility_of_element_located((By.ID, 'post-list')))
    post_list = driver.find_element(By.ID, 'post-list')
    post = post_list.find_element(By.XPATH, f'div[2]/div[1]/span[1]/a')
    post.click()
    
    wait.until(EC.visibility_of_element_located((By.ID, 'right-col')))
    img_spot = driver.find_element(By.ID, 'right-col')
    img = img_spot.find_element(By.XPATH, f'div/img')
    img_URL = img.get_attribute('src')
    if img_URL == d.get('image'):
        print('Image already posted')
    else:
        await documts.document(doc.id).update({'image': img_URL})
        print('Downloading image %s...' % (img_URL))
        urllib.request.urlretrieve(img_URL, os.getcwd() + '/img1.jpg')